In [1]:
import pandas as pd
import numpy as np
from functools import reduce
from IPython.display import display
import re

In [2]:
vaccinationsDF = pd.read_csv('dataFiles/vaccinations.csv', sep=',', header=0)
casesDF = pd.read_csv('dataFiles/covidCases.csv', sep=',', header=0)
iso_codesDF = pd.read_csv('dataFiles/iso_codes.csv', sep=',', header=1)

In [3]:
casesDF.rename(columns={'Date_reported':'date', 'Country':'location', 'Country_code': 'iso2Code'}, inplace=True)

vaccinationsDF.rename(columns={'iso_code': 'iso3Code'}, inplace=True)
vaccinationsDF.drop(vaccinationsDF[(vaccinationsDF['iso3Code'].apply(len) != 3)].index, inplace=True)

iso_codesDF.drop(iso_codesDF.columns[[1, 2, 5, 6, 7]], axis=1, inplace=True)
iso_codesDF.rename(columns={'Country name':'location', 'Alpha-2 code':'iso2Code', 'Alpha-3 code':'iso3Code'}, inplace=True)
iso_codesDF = iso_codesDF.astype(str)
iso_codesDF.drop(iso_codesDF[(iso_codesDF['iso2Code'].apply(len) != 2) | (iso_codesDF['iso3Code'].apply(len) != 3)].index, inplace=True)

In [ ]:
display(casesDF.isnull().sum())

display(casesDF['Country'].where(casesDF['Country_code'].isnull()).unique())
display(vaccinationsDF.isnull().sum())

In [19]:
display(casesDF['location'].where(casesDF['iso2Code'].isnull()).unique())
display(casesDF['iso2Code'].where(casesDF['location'].isnull()).unique())

array([nan, 'Namibia'], dtype=object)

array([nan], dtype=object)

In [9]:
display(len(casesDF.location.unique()))
display(len(vaccinationsDF.location.unique()))
display(len(iso_codesDF.location.unique()))
# display(casesDF.tail(5))
# display(vaccinationsDF.tail(5))

# display(vaccinationsDF)

237

235

248

In [13]:
def common_member(a, b):
    result = [i for i in a if i not in b]
    return result

# display(common_member(casesDF.location.unique(), vaccinationsDF.location.unique()))

# display(common_member(casesDF.iso2Code.unique(), iso_codesDF.iso2Code.unique()))
# display(common_member(vaccinationsDF.iso3Code.unique(), iso_codesDF.iso3Code.unique()))
display(casesDF.iso2Code.unique())

array(['AF', 'AL', 'DZ', 'AS', 'AD', 'AO', 'AI', 'AG', 'AR', 'AM', 'AW',
       'AU', 'AT', 'AZ', 'BS', 'BH', 'BD', 'BB', 'BY', 'BE', 'BZ', 'BJ',
       'BM', 'BT', 'BO', 'XA', 'BA', 'BW', 'BR', 'VG', 'BN', 'BG', 'BF',
       'BI', 'CV', 'KH', 'CM', 'CA', 'KY', 'CF', 'TD', 'CL', 'CN', 'CO',
       'KM', 'CG', 'CK', 'CR', 'CI', 'HR', 'CU', 'CW', 'CY', 'CZ', 'KP',
       'CD', 'DK', 'DJ', 'DM', 'DO', 'EC', 'EG', 'SV', 'GQ', 'ER', 'EE',
       'SZ', 'ET', 'FK', 'FO', 'FJ', 'FI', 'FR', 'GF', 'PF', 'GA', 'GM',
       'GE', 'DE', 'GH', 'GI', 'GR', 'GL', 'GD', 'GP', 'GU', 'GT', 'GG',
       'GN', 'GW', 'GY', 'HT', 'VA', 'HN', 'HU', 'IS', 'IN', 'ID', 'IR',
       'IQ', 'IE', 'IM', 'IL', 'IT', 'JM', 'JP', 'JE', 'JO', 'KZ', 'KE',
       'KI', 'XK', 'KW', 'KG', 'LA', 'LV', 'LB', 'LS', 'LR', 'LY', 'LI',
       'LT', 'LU', 'MG', 'MW', 'MY', 'MV', 'ML', 'MT', 'MH', 'MQ', 'MR',
       'MU', 'YT', 'MX', 'FM', 'MC', 'MN', 'ME', 'MS', 'MA', 'MZ', 'MM',
       nan, 'NR', 'NP', 'NL', 'NC', 'NZ', 'NI', 'NE

In [4]:
df = reduce(lambda left, right: pd.merge(left, right, on=['date', 'iso2Code', 'iso3Code'], how='outer'), [casesDF, vaccinationsDF, iso_codesDF])
df = df.sort_values(by=['date', 'location'])
df = df.sort_values(by=['date', 'location'])
display(df.head(5))
df.to_csv('merged2.csv', header=True, sep=';')

KeyError: 'iso2Code'

In [ ]:
vaccinationsDF['date'] = pd.to_datetime(vaccinationsDF['date'], format='%Y-%m-%d').apply(lambda x: x.toordinal())
casesDF['Date_reported'] = pd.to_datetime(casesDF['Date_reported'], format='%Y-%m-%d').apply(lambda x: x.toordinal())